<a href="https://colab.research.google.com/github/fdac25/trading/blob/main/src/forbesFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
import re
import time

In [2]:
# Google search url
# search_url = "https://www.google.com/search?q=allintitle:+nvidia+site:forbes.com&lr=&sca_esv=827bd599c16e32c5&as_qdr=all&tbs=sbd:1&tbm=nws&ei=kmbtaPyvCeurwbkP3e7UkQo&start=0&sa=N&ved=2ahUKEwj8wO6Sh6KQAxXrVTABHV03NaI4ChDy0wN6BAgFEAQ&biw=2048&bih=1026&dpr=1.25"
# search_url = "https://www.google.com/search?q=allintitle:+nvidia+site:forbes.com+before:2020-01-01+after:2019-01-01&tbs=sbd:1&tbm=nws&start=0"
# https://www.google.com/search?q=allintitle%3Anvidia+site%3Aforbes.com+before%3A2020-01-01+after%3A2019-01-01&sca_esv=183c80e420c11d72&tbs=sbd%3A1&tbm=nws&sclient=gws-wiz-news
def buildSearch(searchString, year, pageNum=0):
  base = "https://www.google.com/search?q="
  search = searchString.replace(" ", "+")
  date = "+before:" + str(year+1) + "-01-01+after:" + str(year) + "-01-01"
  tags = "&tbs=sbd:1&tbm=nws&start=" + str(pageNum * 10)
  return base + search + date + tags

# TEST
print(buildSearch("allintitle:nvidia site:forbes.com", 2018, pageNum=0))

https://www.google.com/search?q=allintitle:nvidia+site:forbes.com+before:2019-01-01+after:2018-01-01&tbs=sbd:1&tbm=nws&start=0


In [ ]:
# NOTE: if this cell shows the html for a CAPTCHA page, the runtime needs to be disconnected, deleted, and reconnected

# Get html from url
response = requests.get(buildSearch("allintitle:nvidia site:forbes.com", 2018))
soup = BeautifulSoup(response.content, 'html.parser')

# Print html
print(soup.prettify())

In [4]:
# Link extraction

# Find first occurance of a char in a string
def find_first(s, c):
  for i in range(len(s)):
    if(s[i] == c): return i
  return -1

# Find all elements containing Forbes news article links
links = soup.find_all(href=re.compile("https://www.forbes.com/sites"))

# Extract news links and titles
extracted_links = []
for link in links:
    href = link.get("href")
    href = href[find_first(href, '=')+1: find_first(href, '&')] # Trim unnecessary parts of link
    text = link.get_text()
    if href: # Ensure the href attribute exists
        extracted_links.append({"href": href, "text": text})

for link_data in extracted_links:
  print(link_data['text'])
  print("  " + str(link_data['href']))
  print(link_data['href'].split('/')[5])
  print("")

NVIDIA Wins First AI BenchmarksForbesIn other words, MLPerf allows results that demonstrate performance at scale as well as single node performance. Once again, NVIDIA wins out, as shown in Figure...82 months ago
  https://www.forbes.com/sites/moorinsights/2018/12/13/nvidia-wins-first-ai-benchmarks/
2018

Nvidia's Monster Titan RTX Has $2500 Price TagForbesNvidia's Monster Titan RTX Has $2500 Price Tag. ByAntony Leather,. Senior Contributor. Forbes contributors publish independent expert analyses and insights.83 months ago
  https://www.forbes.com/sites/antonyleather/2018/12/04/nvidias-monster-titan-rtx-has-2500-price-tag/
2018

Nvidia Surprise: PhysX Is Now An Open Source TechnologyForbesI've long been a critic of Nvidia's “black box” approach to its proprietary graphics libraries. In fact, GameWorks was one of the primary motivators for...83 months ago
  https://www.forbes.com/sites/jasonevangelho/2018/12/04/nvidia-surprise-physx-is-now-an-open-source-technology/
2018

Here's Nvidia'

In [5]:
# Iterate over multiple google search pages
def full_search(searchString, year):
  print(f"Searching in year {year}...")
  extracted_links = []
  max_search = 99

  for i in range(max_search):
    # Build search url
    url = buildSearch(searchString, year, i)

    # Get html from url
    response = requests.get(url)
    raw = response.text
    soup = BeautifulSoup(response.content, 'html.parser')

    # Check to make sure request isn't blocked by CAPTCHA
    is_bad_response = re.match(r'<!DOCTYPE [^>]+>', raw)
    if is_bad_response:
      print("BAD HTML RESPONSE - POSSIBLE CAPTCHA BLOCK")
      print(soup.prettify())
      break;

    # Find all elements containing Forbes news article links
    links = soup.find_all(href=re.compile("https://www.forbes.com/sites"))

    # Extract news links and titles
    for link in links:
        href = link.get("href")
        href = href[find_first(href, '=')+1: find_first(href, '&')]
        text = link.get_text()
        if href and href.split('/')[5] == str(year): # Ensure the href attribute exists and is from the correct year
            extracted_links.append(href)

    # Print status
    print(f"Extracted {len(links)} links from search page {i+1}")
    if len(links) == 0:
      print(f"Page runout - ending search for year {year}")
      print(f"Found {len(extracted_links)} for year {year}")
      return extracted_links
    time.sleep(1)

# TEST
# result = full_search("allintitle:nvidia site:forbes.com", 2018)
# for link in result:
#   print(link)
# print(f"Found {len(result)} links")

In [6]:
# Complete script - iterate over multiple years
year = 2025
links = []

while True:
  new_links = full_search("allintitle:nvidia site:forbes.com", year)
  if len(new_links) > 0: # Add links to list and move on to previous year
    links.extend(new_links)
    year -= 1
  else: # End search if there are no articles found for the year
    print(f"Year runout - ending search")
    break

# # Find and remove duplicates
# num_links = len(extracted_links)
# extracted_links = set(extracted_links)
# num_dup = num_links - len(extracted_links)

# Print results
for link_data in links:
  # print(link_data['text'])
  print("" + str(link_data))
  # print("")
print(f"Found {len(links)} links")


Searching in year 2025...
Extracted 10 links from search page 1
Extracted 10 links from search page 2
Extracted 10 links from search page 3
Extracted 10 links from search page 4
Extracted 10 links from search page 5
Extracted 10 links from search page 6
Extracted 9 links from search page 7
Extracted 10 links from search page 8
Extracted 10 links from search page 9
Extracted 9 links from search page 10
Extracted 10 links from search page 11
Extracted 10 links from search page 12
Extracted 10 links from search page 13
Extracted 10 links from search page 14
Extracted 10 links from search page 15
Extracted 10 links from search page 16
Extracted 10 links from search page 17
Extracted 10 links from search page 18
Extracted 9 links from search page 19
Extracted 10 links from search page 20
Extracted 10 links from search page 21
Extracted 10 links from search page 22
Extracted 10 links from search page 23
Extracted 8 links from search page 24
Extracted 1 links from search page 25
Extracted 0 l

In [7]:
import pandas as pd

# Export links as CSV
df_final = pd.DataFrame(links, columns=['Link'])
df_final.to_csv('forbes_search.csv', index=True)
df_final.head(10)

,Link
0,https://www.forbes.com/sites/catherinebrock/20...
1,https://www.forbes.com/sites/greatspeculations...
2,https://www.forbes.com/sites/johnkoetsier/2025...
3,https://www.forbes.com/sites/ywang/2025/11/05/...
4,https://www.forbes.com/sites/jonmarkman/2025/1...
5,https://www.forbes.com/sites/roomykhan/2025/11...
6,https://www.forbes.com/sites/siladityaray/2025...
7,https://www.forbes.com/sites/marcochiappetta/2...
8,https://www.forbes.com/sites/the-prototype/202...
9,https://www.forbes.com/sites/carminegallo/2025...
